File title basics

In [ ]:
df = pd.read_csv(
    r"D:\Kaggle datasets\title.basics.tsv.gz",
    sep="\t",
    na_values="\\N",
    low_memory=False
)

In [6]:
df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894.0,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892.0,NaN,5,"Animation,Short"
2,tt0000003,short,Poor Pierrot,Pauvre Pierrot,0,1892.0,NaN,5,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892.0,NaN,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893.0,NaN,1,Short


In [16]:
import duckdb
import pandas as pd

file_path = r"D:/Kaggle datasets/title.basics.tsv.gz"

# Load limited rows
query = f"""
SELECT
   tconst,
   titleType,
   primaryTitle,
   originalTitle,
   isAdult,
   startYear,
   endYear,
   runtimeMinutes,
   genres
FROM read_csv_auto('{file_path}', delim='\t', header=True, nullstr='\\N')
LIMIT 10000
"""
df = duckdb.sql(query).df()

# Drop duplicates
df = df.drop_duplicates()

# Strip string columns
for col in df.select_dtypes(include="object").columns:
    df[col] = df[col].str.strip()

# Replace empty strings / \N with pd.NA before converting numeric
df = df.replace(r'^\s*$', pd.NA, regex=True)
df = df.replace('\\N', pd.NA)

# Numeric columns → Int64 (nullable)
numeric_cols = ["isAdult", "startYear", "endYear", "runtimeMinutes"]
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')  # blanks -> NA

# String columns → fill optional
string_cols = ["titleType", "primaryTitle", "originalTitle", "genres"]
df[string_cols] = df[string_cols].fillna("Unknown")

# Save CSV for MySQL → blanks exported as empty (safe for numeric)
df.to_csv(
    "D:/Kaggle datasets/title_basics_clean_mysql.csv",
    index=False,
    quoting=0,    # No quotes
    na_rep=''     # blanks → empty → MySQL will read as NULL
)

print("✅ Cleaned CSV ready for MySQL (numeric blanks safe)")


✅ Cleaned CSV ready for MySQL (numeric blanks safe)


In [20]:
pip install duckdb


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/11.4 MB ? eta -:--:--
    --------------------------------------- 0.3/11.4 MB ? eta -:--:--
   ---- ----------------------------------- 1.3/11.4 MB 4.5 MB/s eta 0:00:03
   ------- -------------------------------- 2.1/11.4 MB 4.5 MB/s eta 0:00:03
   ----------- ---------------------------- 3.1/11.4 MB 4.2 MB/s eta 0:00:02
   -------------- ------------------------- 4.2/11.4 MB 4.2 MB/s eta 0:00:02
   ----------------- ---------------------- 5.0/11.4 MB 4.2 MB/s eta 0:00:02
   -------------------- ------------------- 5.8/11.4 MB 4.1 MB/s eta 0:00:02
   ---------------------- ----------------- 6.3/11.4 MB 3.9 MB/s eta 0:00:02
   ------------------------ --------------- 7.1/11.4 MB 3.8 MB/s eta 0:00:02
   -------------------------- ------------- 7.6/11.4 MB 3.7 MB/s eta 0:00:02
   ---------------------------- ----------- 8.1/11.4 MB 3.6 MB/s eta 0:00:01
   ---------

In [ ]:
!pip install ipywidgets --upgrade

File title akas

In [1]:
import duckdb
import pandas as pd

# Path to your IMDb file
file_path = r"D:/Kaggle datasets/title.akas.tsv.gz"

# Query with LIMIT to avoid memory issues
query = f"""
    SELECT
        titleId,
        ordering,
        title,
        region,
        language,
        types,
        attributes,
        CAST(isOriginalTitle AS INTEGER) AS isOriginalTitle
    FROM read_csv_auto('{file_path}', delim='\t', header=True, nullstr='\\N')
    LIMIT 10000
"""

# Execute query and fetch as pandas DataFrame
df = duckdb.sql(query).df()

# Drop duplicates
df = df.drop_duplicates()

# Save cleaned sample to CSV
output_path = r"D:/Kaggle datasets/title_akas_sample_clean.csv"
df.to_csv(output_path, index=False, encoding='utf-8')

print("✅ Cleaning complete. File saved at:", output_path)


✅ Cleaning complete. File saved at: D:/Kaggle datasets/title_akas_sample_clean.csv


In [2]:
import duckdb
import pandas as pd

# Path to your IMDb file
file_path = r"D:/Kaggle datasets/title.akas.tsv.gz"

# Query with LIMIT to avoid memory issues
query = f"""
    SELECT
        titleId,
        ordering,
        title,
        region,
        language,
        types,
        attributes,
        CAST(isOriginalTitle AS INTEGER) AS isOriginalTitle
    FROM read_csv_auto('{file_path}', delim='\t', header=True, nullstr='\\N')
    LIMIT 10000
"""

# Execute query and fetch as pandas DataFrame
df = duckdb.sql(query).df()

# Drop duplicates
df = df.drop_duplicates()

# -----------------------------
# Handle NULL values
# -----------------------------
# Option 1: Drop rows with any NULLs
# df = df.dropna()

# Option 2: Fill missing values
df = df.fillna({
    "region": "Unknown",
    "language": "Unknown",
    "types": "Unknown",
    "attributes": "None",
    "title": "Untitled"
})

# -----------------------------
# Fix dtypes
# -----------------------------
df["ordering"] = pd.to_numeric(df["ordering"], errors="coerce").astype("Int64")   # integer but allows NaN
df["isOriginalTitle"] = df["isOriginalTitle"].astype("Int64")  # integer 0/1
df["region"] = df["region"].astype("category")
df["language"] = df["language"].astype("category")
df["types"] = df["types"].astype("category")
df["attributes"] = df["attributes"].astype("category")

# -----------------------------
# Save cleaned sample to CSV
# -----------------------------
output_path = r"D:/Kaggle datasets/title_akas_sample_clean.csv"
df.to_csv(output_path, index=False, encoding='utf-8')

print("✅ Cleaning complete. File saved at:", output_path)
print("🔎 Data types after cleaning:\n", df.dtypes)


✅ Cleaning complete. File saved at: D:/Kaggle datasets/title_akas_sample_clean.csv
🔎 Data types after cleaning:
 titleId              object
ordering              Int64
title                object
region             category
language           category
types              category
attributes         category
isOriginalTitle       Int64
dtype: object


File title crew

In [7]:
import pandas as pd

df = pd.read_csv(
    r"D:\Kaggle datasets\title.crew.tsv.gz",
    sep="\t",
    na_values="\\N",
    low_memory=False
)

# Now preview
print(df.head())


      tconst  directors    writers
0  tt0000001  nm0005690        NaN
1  tt0000002  nm0721526        NaN
2  tt0000003  nm0721526  nm0721526
3  tt0000004  nm0721526        NaN
4  tt0000005  nm0005690        NaN


In [12]:
import duckdb
import pandas as pd

# Path to your IMDb file
file_path = r"D:/Kaggle datasets/title.crew.tsv.gz"

# Query with LIMIT to avoid memory issues
query = f"""
    SELECT
        tconst,
        directors,
        writers,
    FROM read_csv_auto('{file_path}', delim='\t', header=True, nullstr='\\N')
    LIMIT 10000
"""

# Execute query and fetch as pandas DataFrame
df = duckdb.sql(query).df()

# Drop duplicates
df = df.drop_duplicates()

# -----------------------------
# Handle NULL values
# -----------------------------
# Option 1: Drop rows with any NULLs
# df = df.dropna()

# Option 2: Fill missing values
df = df.fillna({
    "directors": "Unknown",
    "writers": "Unknown",
})

# -----------------------------
# Fix dtypes
# -----------------------------
# df["ordering"] = pd.to_numeric(df["ordering"], errors="coerce").astype("Int64")   # integer but allows NaN
# df["isOriginalTitle"] = df["isOriginalTitle"].astype("Int64")  # integer 0/1
# df["region"] = df["region"].astype("category")
# df["language"] = df["language"].astype("category")
# df["types"] = df["types"].astype("category")
# df["attributes"] = df["attributes"].astype("category")

# -----------------------------
# Save cleaned sample to CSV
# -----------------------------
output_path = r"D:/Kaggle datasets/title_crew_sample_clean.csv"
df.to_csv(output_path, index=False, encoding='utf-8')

print("✅ Cleaning complete. File saved at:", output_path)
# print("🔎 Data types after cleaning:\n", df.dtypes)


✅ Cleaning complete. File saved at: D:/Kaggle datasets/title_crew_sample_clean.csv


File title episodes

In [13]:
import pandas as pd

df = pd.read_csv(
    r"D:\Kaggle datasets\title.episode.tsv.gz",
    sep="\t",
    na_values="\\N",
    low_memory=False
)

# Now preview
print(df.head())


      tconst parentTconst  seasonNumber  episodeNumber
0  tt0031458   tt32857063           NaN            NaN
1  tt0041951    tt0041038           1.0            9.0
2  tt0042816    tt0989125           1.0           17.0
3  tt0042889    tt0989125           NaN            NaN
4  tt0043426    tt0040051           3.0           42.0


In [17]:
import duckdb
import pandas as pd

# Path to your IMDb file
file_path = r"D:/Kaggle datasets/title.episode.tsv.gz"

# Query with LIMIT to avoid memory issues
query = f"""
    SELECT
        tconst,
        parentTconst,
        seasonNumber,
        episodeNumber,
    FROM read_csv_auto('{file_path}', delim='\t', header=True, nullstr='\\N')
    LIMIT 10000
"""

# Execute query and fetch as pandas DataFrame
df = duckdb.sql(query).df()

# Drop duplicates
df = df.drop_duplicates()

# -----------------------------
# Handle NULL values
# -----------------------------
# Option 1: Drop rows with any NULLs
df = df.dropna()

# Option 2: Fill missing values
# df = df.fillna({
#     "seasonNumber": "Null",
#     "wepisodeNumber": "Null",
# })

# -----------------------------
# Fix dtypes
# -----------------------------
# df["ordering"] = pd.to_numeric(df["ordering"], errors="coerce").astype("Int64")   # integer but allows NaN
# df["isOriginalTitle"] = df["isOriginalTitle"].astype("Int64")  # integer 0/1
# df["region"] = df["region"].astype("category")
# df["language"] = df["language"].astype("category")
# df["types"] = df["types"].astype("category")
# df["attributes"] = df["attributes"].astype("category")

# -----------------------------
# Save cleaned sample to CSV
# -----------------------------
output_path = r"D:/Kaggle datasets/title_episode_sample_clean.csv"
df.to_csv(output_path, index=False, encoding='utf-8')

print("✅ Cleaning complete. File saved at:", output_path)
# print("🔎 Data types after cleaning:\n", df.dtypes)


✅ Cleaning complete. File saved at: D:/Kaggle datasets/title_episode_sample_clean.csv


File title ratings

In [19]:
import pandas as pd

df = pd.read_csv(
    r"D:\Kaggle datasets\title.ratings.tsv.gz",
    sep="\t",
    na_values="\\N",
    low_memory=False
)

# Now preview
print(df.head())


      tconst  averageRating  numVotes
0  tt0000001            5.7      2178
1  tt0000002            5.5       299
2  tt0000003            6.4      2243
3  tt0000004            5.2       193
4  tt0000005            6.2      2988


In [6]:
import duckdb
import pandas as pd

# Path to your IMDb file
file_path = r"D:/Kaggle datasets/title.ratings.tsv.gz"

# Query with LIMIT to avoid memory issues
query = f"""
    SELECT
        tconst,
        averageRating,
        numVotes,
    FROM read_csv_auto('{file_path}', delim='\t', header=True, nullstr='\\N')
    LIMIT 10000
"""

# Execute query and fetch as pandas DataFrame
df = duckdb.sql(query).df()

# Drop duplicates
df = df.drop_duplicates()

# -----------------------------
# Handle NULL values
# -----------------------------
# Option 1: Drop rows with any NULLs
df = df.dropna()

# Option 2: Fill missing values
# df = df.fillna({
#     "seasonNumber": "Null",
#     "wepisodeNumber": "Null",
# })

# -----------------------------
# Fix dtypes
# -----------------------------
# df["ordering"] = pd.to_numeric(df["ordering"], errors="coerce").astype("Int64")   # integer but allows NaN
# df["isOriginalTitle"] = df["isOriginalTitle"].astype("Int64")  # integer 0/1
# df["region"] = df["region"].astype("category")
# df["language"] = df["language"].astype("category")
# df["types"] = df["types"].astype("category")
# df["attributes"] = df["attributes"].astype("category")

# -----------------------------
# Save cleaned sample to CSV
# -----------------------------
output_path = r"D:/Kaggle datasets/title_ratings_sample_clean.csv"
df.to_csv(output_path, index=False, encoding='utf-8')

print("✅ Cleaning complete. File saved at:", output_path)
# print("🔎 Data types after cleaning:\n", df.dtypes)


✅ Cleaning complete. File saved at: D:/Kaggle datasets/title_ratings_sample_clean.csv


In [ ]:
File title principals

In [1]:
import pandas as pd

df = pd.read_csv(
    r"D:\Kaggle datasets\title.principals.tsv.gz",
    sep="\t",
    na_values="\\N",
    low_memory=False
)

# Now preview


In [3]:
df.tail()

,tconst,ordering,nconst,category,job,characters
94747062,tt9916880,17,nm0996406,director,principal director,NaN
94747063,tt9916880,18,nm1482639,writer,NaN,NaN
94747064,tt9916880,19,nm2586970,writer,books,NaN
94747065,tt9916880,20,nm1594058,producer,producer,NaN
94747066,tt9916880,21,nm1482639,producer,producer,NaN


In [8]:
import duckdb
import pandas as pd

file_path = r"D:/Kaggle datasets/title.principals.tsv.gz"

# Query with LIMIT to avoid memory issues
query = f"""
    SELECT
        tconst,
        ordering,
        nconst,
        category,
        job,
        characters
    FROM read_csv_auto('{file_path}', delim='\t', header=True, nullstr='\\N')
    LIMIT 10000
"""

# Execute query and fetch as pandas DataFrame
df = duckdb.sql(query).df()

# Drop duplicates
df = df.drop_duplicates()

# -----------------------------
# Handle NULL values
# -----------------------------
# Option 1: Drop rows with all NULLs in critical columns
df = df.dropna(subset=['tconst', 'nconst'])  # optional
# Option 2: Fill missing values (if needed)
# df = df.fillna({"job": "Unknown", "characters": "Unknown"})

# -----------------------------
# Fix dtypes
# -----------------------------
df["ordering"] = pd.to_numeric(df["ordering"], errors="coerce").astype("Int64")  # nullable int
df["category"] = df["category"].astype("category")
df["job"] = df["job"].astype("string")
df["characters"] = df["characters"].astype("string")

# -----------------------------
# Save cleaned sample to CSV
# -----------------------------
output_path = r"D:/Kaggle datasets/title_principal_sample_clean.csv"
df.to_csv(output_path, index=False, encoding='utf-8')

print("✅ Cleaning complete. File saved at:", output_path)
print("🔎 Data types after cleaning:\n", df.dtypes)


✅ Cleaning complete. File saved at: D:/Kaggle datasets/title_principal_sample_clean.csv
🔎 Data types after cleaning:
 tconst                object
ordering               Int64
nconst                object
category            category
job           string[python]
characters    string[python]
dtype: object
